# 7.1 Contour Feature
- There are several different features of contours, like area, perimeter, centroid, bounding box etc.
    - 7.1.1 Contour Area
    - 7.1.2 Bonding Rectangle
    - 7.1.3 Contour Perimeter
    - 7.1.4 Contour Approximation
    <br><br>

In [ ]:
import cv2
import numpy as np

<br><br>
### 7.1.1 **Contour Area (luasan)** <br>
`area = cv2.contourArea(contour)`<br><br>

- Example of find contours and contour area calculation on `block_2.png` image

In [ ]:
# read the image "blocks_2.jpg"
img = cv2.imread("blocks_2.jpg")

# convert BGR image to HSV
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# apply inverse binary thresholding at threshold value 90
__, binary_img = cv2.threshold(gray, 235, 255, cv2.THRESH_BINARY_INV)

# find contours and hierarchy  
# retrive only external contours (cv2.RETR_EXTERNAL) with all points (cv2.CHAIN_APPROX_NONE)
contours, __ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)


for contour in contours:
    # calculate contour area (number of pixels inside the contour)
    area = cv2.contourArea(contour)

    print("Contour Area: ", area)

    # draw all contours in green color with thickness of 2 px
    cv2.drawContours(img, contour, -1, (0, 255, 0), 2)



# display images
cv2.imshow('original image', img)
cv2.imshow('binary image', binary_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

- Contour area can be used to <font color="orange">filter out small contours</font> that may be noise in the image. 
- For example, you can set a threshold area value and only <font color="orange">keep contours with an area larger</font> than that threshold.<br><br>

- Example of filtering small contours based on area on `tomatoes_2.jpg` image

In [ ]:
# define thresholding range for below 0 hue red in HSV space
red_1_lower = np.array([162, 160, 80])
red_1_upper = np.array([179, 255, 255])

# define thresholding range for above 0 hue red in HSV space
red_2_lower = np.array([0, 160, 80])
red_2_upper = np.array([8, 255, 255])


# read the image "tomatoes_2.jpg"
img = cv2.imread("tomatoes_2.jpg")


# apply gaussian blur with 15x15 kernel size with sigmaX=0 and sigmaY=0
# bigger kernel size means more blur effect
gaussian_blur_img = cv2.GaussianBlur(img, (17, 17), 0, 0)


# convert BGR image to HSV Image
hsv_img = cv2.cvtColor(gaussian_blur_img, cv2.COLOR_BGR2HSV)


# apply range thresholding for both red ranges
mask_1_img = cv2.inRange(hsv_img, red_1_lower, red_1_upper)
mask_2_img = cv2.inRange(hsv_img, red_2_lower, red_2_upper)

# combine both mask images using addition operation
mask_img = mask_1_img + mask_2_img

# find contour from mask image using RETR_EXTERNAL method
contours, hierarchy = cv2.findContours(mask_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)


# filter out small contours based on area
min_area = 1000 
filtered_contours = [] 

for contour in contours:
    area = cv2.contourArea(contour)

    # filter based on area, keep only contours with area >= min_area
    if area >= min_area:

        filtered_contours.append(contour)
        cv2.drawContours(img, contour, -1, (255, 0, 0), 2)


cv2.putText(img, 
            "Contour Count: " + str(len(filtered_contours)), 
            (10, 30), 	                   
            cv2.FONT_HERSHEY_SIMPLEX,     
            1,                          
            (0, 255, 255),
            2)



# display images
cv2.imshow('original image', img)
cv2.imshow('mask image', mask_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

<br><br><br>
### 7.1.2 **Bounding Rectangle** <br>
<img src="res/boundingrect.png" style="width:200px; margin-top:10px;"></img>
- **Straight Bounding Rectangle** <font color="green">(green box)</font> <br>
it doesn't consider the rotation of the object. So area of the bounding rectangle won't be minimum. <br>
`x, y, w, h = cv2.boundingRect(contour)`<br>
where `x,y` is top-left point of rectangle, and `w,h` are width and height of rectangle.<br><br>
- **Rotated Rectangle** <font color="red">(red box)</font> <br>
drawn with minimum area, so it considers the rotation also.<br>
`(x,y), (w, h), angle = cv2.minAreaRect(contour)`<br>
where `(x,y)` is center point of rectangle, `w, h` are width and height of rectangle, and `angle` is angle of rotation. <font color="orange">Angle of rotation</font> is relative to the <font color="orange">positive X-axis</font>, measured <font color="orange">clockwise</font>.<br><br><br>

- example of finding bounding rectangle on `blocks_2.jpg` image

In [ ]:
# read the image "blocks_2.jpg"
img = cv2.imread("blocks_2.jpg")


# convert BGR image to HSV
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# apply inverse binary thresholding at threshold value 90
__, binary_img = cv2.threshold(gray, 235, 255, cv2.THRESH_BINARY_INV)

# find contours and hierarchy  
# retrive only external contours (cv2.RETR_EXTERNAL) with all points (cv2.CHAIN_APPROX_NONE)
contours, __ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

for contour in contours:

    # find bounding rectangle
    x, y, w, h = cv2.boundingRect(contour)


    print("Bounding Rect: x=", x, " y=", y, " w=", w, " h=", h)

    # draw bounding rectangle in purple color with thickness of 2 px
    cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 255), 2)

    

# display images
cv2.imshow('original image', img)
cv2.imshow('binary image', binary_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

- bounding rectangle can be used for <font color="orange">object localization</font> in an image. 
- For example, in object detection tasks, bounding rectangles are often used to <font color="orange">define the location of detected objects</font> within an image.

<br><br><br>
### 7.1.3 **Contour Perimeter (keliling)** <br>
`perimeter = cv2.arcLength(contour, closed)`<br>
- Where `closed` is a boolean value indicating whether the contour is closed or not. 
    - If it is closed, set it to `True`, otherwise `False`.
- returns float value indicating the perimeter length in pixels.<br>

- Example of find contours and contour perimeter calculation on `blocks_2.jpg` image

In [ ]:
# read the image "blocks_2.jpg"
img = cv2.imread("blocks_2.jpg")


# convert BGR image to HSV
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# apply inverse binary thresholding at threshold value 90
__, binary_img = cv2.threshold(gray, 235, 255, cv2.THRESH_BINARY_INV)

# find contours and hierarchy  
# retrive only external contours (cv2.RETR_EXTERNAL) with all points (cv2.CHAIN_APPROX_NONE)
contours, __ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

for contour in contours:

    # calculate contour perimeter (keliling kontur)
    perimeter = cv2.arcLength(contour, True)
    print("Contour Perimeter: ", perimeter)

    # draw all contours in green color with thickness of 3 px
    cv2.drawContours(img, contour, -1, (255, 0, 255), 2)



# display images
cv2.imshow('original image', img)
cv2.imshow('binary image', binary_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

- Contour perimeter can be useful in <font color="orange">shape analysis and object recognition tasks.</font>
- it can be combined with `cv2.approxPolyDP()` function to approximate the contour shape to a polygon with fewer vertices, which can be useful for shape simplification and recognition.

<br><br><br>
### 7.1.4 **Contour Approximation** <br>
It approximates a contour shape to another shape with <font color="orange">less number of vertices</font> depending upon the precision we specify.<br><br> 
`epsilon = fraction * cv2.arcLength(contour, closed)` <br>
`approx = cv2.approxPolyDP(contour, epsilon, closed)` <br><br>
`fraction` is a value between 0 and 1 that determines the approximation accuracy.<br><br>
Below, in second image, green line shows the approximated curve for epsilon = 10% (0.1) of arc length. <br>
Third image shows the same for epsilon = 1% (0.01) of the arc length. <br>
<img src="res/approx.jpg" style="width:400px; margin-top:10px;"></img><br><br><br>

- Example of contour approximation on `blocks_2.jpg` image

In [ ]:
# read the image "blocks_2.jpg"
img = cv2.imread("blocks_2.jpg")

# convert BGR image to HSV
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# apply inverse binary thresholding at threshold value 90
__, binary_img = cv2.threshold(gray, 235, 255, cv2.THRESH_BINARY_INV)

# find contours
contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

for contour in contours:

    # approximate contour to polygon with epsilon = 1% (0.01) of arc length
    epsilon = 0.01 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # Number of vertices of approximated polygon
    vertices = len(approx)

    # Shape classification based on number of vertices
    if vertices == 3:
        shape = "Triangle" # 3 sides

    elif vertices == 4:
        shape = "Quadrilateral" # 4 sides, can be square or rectangle

    elif vertices > 4:
        shape = "Polygon" # more than 4 sides

    else:
        shape = "Unknown"

    # Draw approximated contour (polygon)
    cv2.drawContours(img, [approx], -1, (255, 0, 255), 2)

    # find bounding rectangle, to position the shape name text
    x, y, w, h = cv2.boundingRect(approx)

    # Draw shape name
    cv2.putText(
        img,
        shape,
        (x, y - 5),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.4,
        (255, 0, 0),
        1
    )

# display images
cv2.imshow('original image', img)
cv2.imshow('binary image', binary_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

<br><br><br>
- Example improved shape detection with more <font color=orange> accurate distinction</font> between <font color=orange>square and rectangle</font>, <font color=orange>polygon and circle</font> on `blocks_2.jpg` image

In [ ]:
# read the image "blocks_2.jpg"
img = cv2.imread("blocks_2.jpg")

# convert BGR image to HSV
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# apply inverse binary thresholding at threshold value 90
__, binary_img = cv2.threshold(gray, 235, 255, cv2.THRESH_BINARY_INV)

# find contours
contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

for contour in contours:
    # calculate contour area
    area = cv2.contourArea(contour)

    # approximate contour to polygon with epsilon = 1% (0.01) of arc length
    perimeter = cv2.arcLength(contour, True)
    epsilon = 0.01 * perimeter
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # Number of vertices of approximated polygon
    vertices = len(approx)


    # Shape classification based on number of vertices
    if vertices == 3: # 3 sides
        shape = "Triangle" 


    elif vertices == 4: # 4 sides
        x, y, w, h = cv2.boundingRect(approx)

        # if area is close to minimum area, then it's square or rectangle
        if (area / (w*h)) > 0.9:

            # if width and height are almost equal, then it's square
            if w/h >= 0.95 and w/h <= 1.05: 
                shape = "Square"
            else:
                shape = "Rectangle"

        else :
            shape = "Quadrilateral" # trapezoid, kite, skewed quad, etc.


    elif vertices > 4: # more than 4 sides
        # compute circularity, if value is between 0 and 1, where 1 means perfect circle
        circularity = 4 * np.pi * area / (perimeter * perimeter)

        if circularity > 0.85: # close to 1, then it's circle
            shape = "Circle" 
        else:
            shape = "Polygon"

    else:
        shape = "Unknown"

    # Draw approximated contour (polygon)
    cv2.drawContours(img, [approx], -1, (255, 0, 255), 2)

    # find bounding rectangle, to position the shape name text
    x, y, w, h = cv2.boundingRect(approx)

    # Draw shape name
    cv2.putText(
        img,
        shape,
        (x, y - 5),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.4,
        (255, 0, 0),
        1
    )

# display images
cv2.imshow('original image', img)
cv2.imshow('binary image', binary_img)
cv2.waitKey(0)
cv2.destroyAllWindows()